In [1]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import stem
import math
import operator
from gensim.models import Word2Vec
import collections
from pyemd import emd
import operator

Assim como anteriormente, o tratamento utilizado foi a filtragem de palavras, a fim de eliminar caracteres estranhos. Além disso foram removidas as stopwords e todas as palavras possuem apenas letras minúsculas, por compatibilidade à biblioteca. 

In [2]:
# carregando o modelo
model = Word2Vec.load('pt.bin')

# carregando os dados
url = "https://raw.githubusercontent.com/LiviaCavalcanti/RecInfo/master/lab02/results.csv"
original_data = pd.read_csv(url)
N_TOTAL_DOCUMENTS = original_data.shape[0]
original_data.head()

/home/livia/Documentos/miniconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/home/livia/Documentos/miniconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


In [3]:

# para o first time run
nltk.download('stopwords')

# removendo stopwords
default_stopwords = set(stopwords.words('portuguese'))
stopwords_complement = ["", "neste","és", "é", "somos", "sois", "são", "era", "eras", "éreis", "foste", "fostes", "foras",	"fôreis",	"serás", "sereis", "serão",	"serias", "seríeis", "sejas", "sejais", "fosses", "fôsseis", "fores", "fordes", "sê", "sejas", "sejais", "ser", "seres", "sermos",	"serdes",	"serem", "sobre", "é"]

stopwords_complement.extend(default_stopwords)
stopwords_complement.extend(default_stopwords)



[nltk_data] Downloading package stopwords to /home/livia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Implemente uma função que recebe uma notícia e retorna os vetores (word embeddings) das palavras do título dessa notícia a partir dos word embeddings pré-treinados com o gensim (30 pts).**

A tarefa foi dividida em duas funções: uma para a seleção e o tratamento das notícias e outra para retornar o array tratado bem como uma dicionário com os *word embeddings* correspondendes.

In [4]:
def news2array(news_id):
    words = re.split(r'\W+', original_data.loc[news_id].title)
    no_stopwords_news = []
    for word in words:
        if word.lower() not in stopwords_complement:
            no_stopwords_news.append(word.lower())
    return no_stopwords_news

In [5]:
def news2embeddings(news_id):
    words = news2array(news_id)
    
    embed_words = {}
    for word in words:
                
        try:
            embed_words[w] = model[w]
        except:
            continue      
        
    return words, embed_words

**Implemente uma função que calcula o WMD (Word Mover's Distance) entre duas notícias usando os embeddings das palavras dos respectivos títulos  (35 pts).**

Reutilizando o código anterior, no tocante ao tratamento do id da notícia para a obtenção do array de palavras a serem consideradas para o cálculo da distância, junto à função oferecida pelo gensim.

In [6]:
def wmdistance(news1, news2):
    n1, n1_embed = news2embeddings(news1)
    n2, n2_embed = news2embeddings(news2)
    
    return model.wmdistance(n1, n2)

**Implemente uma função que possa receber qualquer notícia como entrada e retornar as top-3 notícias mais similares (menos distantes) a ela (35 pts).**

Por fim, as funções anteriores foram usadas para gerar um ranking de distâncias entre os títulos das notícias passando como entrada seus respectivos ids. Como resultado, é possível ver quais notícias são mais similares a uma nóticia alvo.

In [7]:
def similarity(news_id):
    score = {}
    for i in original_data.index:
        score[i] = wmdistance(news_id, i)
    
    sorted_news = sorted(score.items(), key=operator.itemgetter(1))
    return [(i, original_data.loc[i].title, original_data.loc[i].subtitle, "{0:.2f}".format(j)) for i, j in sorted_news[1:4]]

Como teste, podemos ver que para uma notícia de futebol europeu com foco em times italianos e em Cristiano Ronaldo, tem como notícias similares outras que falam sobre o jogador e outros jogos do campeonato mais importante do continente em questão, a Champions.

In [8]:
original_data.loc[40].title, original_data.loc[40].subtitle

('Turim se apaixona por Cristiano Ronaldo',
 'Juventus o blindou diante do caso de suposto estupro  e ele oferece ao time sua melhor versão.')

In [9]:
similarity(40)

/home/livia/Documentos/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """


[(242,
  'A fortuna blindada de Cristiano Ronaldo',
  'Atacante criou um império empresarial em cuja direção colocou sua família e um círculo muito íntimo de amigos',
  '11.75'),
 (52,
  'Comandado por Messi  Barcelona goleia o Lyon e se classifica na Champions',
  'Argentino faz dois gols e dá duas assistências em goleada por 5 a 1 no Camp Nou. Coutinho  Piqué e Dembele também marcaram',
  '18.68'),
 (50,
  'Cristiano Ronaldo faz três no Atlético de Madrid e conduz Juventus às quartas da Champions',
  'Italianos haviam perdido por 2 a 0 na Espanha  mas reverteram o placar em Turim com uma atuação épica do português',
  '18.74')]

Para uma notícia mais específica, com tema menos comum, sobre problemas Michael Jackson, retorna notícias sobre o artista e sobre outro cantor estadunidense famoso, Eric Clapton. Mas retorna também outra notícia menos correlata evolvemendo jogador brasileiro no país vizinho da Bolívia.

In [10]:
original_data.loc[4].title, original_data.loc[4].subtitle

('Rádios canadenses banem músicas de Michael Jackson da programação',
 'Quebec Cogeco Media toma a decisão após queixas de ouvintes indignados com documentário sobre supostos abusos sexuais cometidos pelo cantor')

In [11]:
similarity(4)

/home/livia/Documentos/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """


[(179,
  '“Nunca vou entender por que deixaram Michael Jackson dormir com crianças”',
  'Diretor de ‘Deixando Neverland’ explica como lida com a reação furiosa que seu documentário tem provocado pelo mundo',
  '21.16'),
 (194,
  '‘Tears in Heaven’: a tragédia que inspirou uma das canções mais comoventes da história',
  'Março é uma época amarga para Eric Clapton. No final desse mês  em 1991  o músico perdeu seu filho de quatro anos num acidente no quarto de um hotel. Pouco tempo depois  dedicou-lhe uma dolorosa melodia. Esta é a história.',
  '21.68'),
 (91,
  'O brasileiro insensato que rebate racismo com xenofobia',
  'Jogador Serginho foi vítima de ofensas racistas e abandonou o campo na Bolívia. Por aqui  alguns torcedores preferem ofender o país vizinho a refletir sobre seus próprios preconceitos',
  '22.99')]